In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("SparkHomework").getOrCreate()

spark

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.bucketing.enabled", "1")

maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv").select(col("mapid"), col("name").alias("map_name"), col("description").alias("map_description"))
medals = spark.read.option("header","true").csv("/home/iceberg/data/medals.csv")
medals_matches_players = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")
matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")


In [5]:
print(maps.count())
maps.show()

40
+--------------------+-------------------+--------------------+
|               mapid|           map_name|     map_description|
+--------------------+-------------------+--------------------+
|c93d708f-f206-11e...|              Urban|Andesia was the c...|
|cb251c51-f206-11e...|     Raid on Apex 7|This unbroken rin...|
|c854e54f-f206-11e...|March on Stormbreak|                NULL|
|c8d69870-f206-11e...| Escape from A.R.C.|Scientists flocke...|
|73ed1fd0-45e5-4bb...|             Osiris|                NULL|
|96c3e3dd-7703-408...|          Blue Team|                NULL|
|1c4f8e19-b046-4f7...|            Glassed|                NULL|
|825065cf-df57-42e...|        Unconfirmed|                NULL|
|9a188f67-1664-4d7...|           Alliance|                NULL|
|2702ea83-2c3e-4fd...|   Before the Storm|                NULL|
|82f8471c-a2ef-408...|            Genesis|                NULL|
|fcd7caa4-37c9-436...|       The Breaking|                NULL|
|7dc80b62-dd39-41d...|          Guard

In [7]:
print(medals_matches_players.count())
medals_matches_players.show()

755229
+--------------------+---------------+----------+-----+
|            match_id|player_gamertag|  medal_id|count|
+--------------------+---------------+----------+-----+
|009fdac5-e15c-47c...|       EcZachly|3261908037|    7|
|009fdac5-e15c-47c...|       EcZachly| 824733727|    2|
|009fdac5-e15c-47c...|       EcZachly|2078758684|    2|
|009fdac5-e15c-47c...|       EcZachly|2782465081|    2|
|9169d1a3-955c-4ea...|       EcZachly|3001183151|    1|
|9169d1a3-955c-4ea...|       EcZachly|3565443938|    6|
|9169d1a3-955c-4ea...|       EcZachly|3491849182|    1|
|4a078b2f-65eb-4c6...|       EcZachly|3261908037|    8|
|9169d1a3-955c-4ea...|       EcZachly|2105198095|    6|
|9169d1a3-955c-4ea...|       EcZachly|2916014239|    3|
|9169d1a3-955c-4ea...|       EcZachly|3261908037|    6|
|9169d1a3-955c-4ea...|       EcZachly|1351381581|    2|
|9169d1a3-955c-4ea...|       EcZachly|2838259753|    1|
|9169d1a3-955c-4ea...|       EcZachly|3354395650|    1|
|9169d1a3-955c-4ea...|       EcZachly| 29

In [8]:
print(matches.count())
matches.show()

24025
+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|     completion_date|match_duration|game_mode|      map_variant_id|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+
|11de1a94-8d07-416...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|         true|2016-02-22 00:00:...|          NULL|     NULL|                NULL|
|d3643e71-3e51-43e...|cb914b9e-f206-11e...|       false|d0766624-dbd7-453...|257a305e-4dd3-41f...|         true|2016-02-14 00:00:...|          NULL|     NULL|                NULL|
|d78d2aae-36e4-48a...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408

In [9]:
print(medals.count())
medals.show()


183
+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+--------------+----------+
|  medal_id|          sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|   classification|         description|          name|difficulty|
+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+--------------+----------+
|2315448068|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|             NULL|                NULL|          NULL|      NULL|
|3565441934|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|             NULL|                NULL|          NULL|      NULL|
|4162659350|https://content.h...|        750|       750|

In [10]:
from pyspark.sql.functions import bucket


matches = spark.read.option("header", "true").option("inferSchema", "true").csv("/home/iceberg/data/matches.csv")
# matches.show()


In [11]:
%%sql
DROP TABLE IF EXISTS bootcamp.matches_bucketed

25/11/01 03:59:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [12]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    mapid STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    game_variant_id STRING,
    is_match_over BOOLEAN,
    completion_date TIMESTAMP,
    match_duration STRING,
    game_mode STRING,
    map_variant_id STRING
  ) USING iceberg PARTITIONED BY (bucket(16, match_id))

++
||
++
++

In [13]:
matches.write.mode("append").bucketBy(16,"match_id").saveAsTable("bootcamp.matches_bucketed")

In [14]:
%%sql
SELECT
  COUNT(1)
FROM
  bootcamp.matches_bucketed.files

count(1)
16


In [15]:

match_details = spark.read.option("header","true").option("inferSchema", "true").csv("/home/iceberg/data/match_details.csv")
match_details.show()


25/11/01 03:59:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+
|            match_id|player_gamertag|previous_spartan_rank|spartan_rank|previous_total_xp|total_xp|previous_csr_tier|previous_csr_designation|previous_csr|previous_csr_percent_to_next_tier|previous_csr_rank|current_

In [16]:
%%sql
DROP TABLE IF EXISTS bootcamp.match_details_bucketed

++
||
++
++

In [17]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.match_details_bucketed (
    match_id STRING,
    player_gamertag STRING,
    previous_spartan_rank INTEGER,
    spartan_rank INTEGER,
    previous_total_xp INTEGER,
    total_xp INTEGER,
    previous_csr_tier STRING,
    previous_csr_designation STRING,
    previous_csr INTEGER,
    previous_csr_percent_to_next_tier STRING,
    previous_csr_rank INTEGER,
    current_csr_tier STRING,
    current_csr_designation STRING,
    current_csr INTEGER,
    current_csr_percent_to_next_tier STRING,
    current_csr_rank INTEGER,
    player_rank_on_team STRING,
    player_finished BOOLEAN,
    player_average_life STRING,
    player_total_kills INTEGER,
    player_total_headshots INTEGER,
    player_total_weapon_damage FLOAT,
    player_total_shots_landed INTEGER,
    player_total_melee_kills INTEGER,
    player_total_melee_damage FLOAT,
    player_total_assassinations INTEGER,
    player_total_ground_pound_kills INTEGER,
    player_total_shoulder_bash_kills INTEGER,
    player_total_grenade_damage FLOAT,
    player_total_power_weapon_damage FLOAT,
    player_total_power_weapon_grabs INTEGER,
    player_total_deaths INTEGER,
    player_total_assists INTEGER,
    player_total_grenade_kills INTEGER,
    did_win INTEGER,
    team_id STRING
  ) USING iceberg PARTITIONED BY (bucket(16, match_id));

++
||
++
++

In [18]:
match_details.write.mode("append").bucketBy(16,"match_id").saveAsTable("bootcamp.match_details_bucketed")

In [19]:
%%sql
SELECT
  SUM(file_size_in_bytes) AS size,
  COUNT(1) AS num_files,
  'sorted'
FROM
  bootcamp.match_details_bucketed.files

size,num_files,sorted
7136552,16,sorted


In [20]:

medals_matches_players = spark.read.option("header","true").option("inferSchema", "true").csv("/home/iceberg/data/medals_matches_players.csv")
# medals_matches_players.show()

In [21]:
%%sql
DROP TABLE IF EXISTS bootcamp.medals_matches_players_bucketed

++
||
++
++

In [22]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.medals_matches_players_bucketed (
    match_id STRING,
    player_gamertag STRING,
    medal_id STRING,
    COUNT INTEGER
  ) USING iceberg PARTITIONED BY (bucket(16, match_id))

++
||
++
++

In [23]:
medals_matches_players.write.mode("append").bucketBy(16,"match_id").saveAsTable("bootcamp.medals_matches_players_bucketed")

In [24]:
%%sql
SELECT
  SUM(file_size_in_bytes) AS size,
  COUNT(1) AS num_files,
  'sorted'
FROM
  bootcamp.medals_matches_players_bucketed.files

size,num_files,sorted
2912283,16,sorted


In [35]:
match_details_bucketed = spark.table("bootcamp.match_details_bucketed")
matches_bucketed = spark.table("bootcamp.matches_bucketed")
medals_matches_players_bucketed = spark.table("bootcamp.medals_matches_players_bucketed").select(col("match_id"), col("player_gamertag").alias("mmp_palyer_gamertag"), col("medal_id"))

In [36]:
from pyspark.sql.functions import broadcast

matches_bucketed_with_maps = matches_bucketed.join(broadcast(maps),"mapid")
matches_bucketed_with_maps.show()

+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+--------------+---------+--------------------+----------------+--------------------+
|               mapid|            match_id|is_team_game|         playlist_id|     game_variant_id|is_match_over|    completion_date|match_duration|game_mode|      map_variant_id|        map_name|     map_description|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+--------------+---------+--------------------+----------------+--------------------+
|ce1dc2de-f206-11e...|f44c9997-eb6f-4d6...|        true|0504ca3c-de41-48f...|b0df8938-0fb6-42e...|         true|2016-02-28 00:00:00|          NULL|     NULL|d5a6277a-96d5-499...|           Truth|                NULL|
|cbcea2c0-f206-11e...|f0f2daf2-52f3-4ff...|        NULL|2323b76a-db98-4e0...|257a305e-4dd3-41f...|         NULL|2016-02-04 00:00:00|

In [37]:

df = match_details_bucketed.join(matches_bucketed_with_maps, "match_id").join(medals_matches_players_bucketed,"match_id").join(broadcast(medals),"medal_id")
df.show();
df.explain()

+----------+--------------------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+--------------------+------------+--------------------+--------------------+-------------+-------------------+--------------+---------+--------------+--------+--------------------+-------------------+------

In [38]:
# Which player averages the most kills per game?
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

aggregated = df.groupBy("match_details_bucketed.player_gamertag").agg(F.sum("player_total_kills").alias("total_kills"))
player_kills = aggregated.orderBy(desc("total_kills"))
player_kills.show()

player_with_most_kills = player_kills.first()["player_gamertag"] 
player_with_most_kills

+---------------+-----------+
|player_gamertag|total_kills|
+---------------+-----------+
|       EcZachly|    1503498|
|    ILLICIT 117|    1020377|
|  JakeWilson801|     532473|
|      Amplafied|     500430|
| Ash All Mighty|     461267|
|  Mark SIII 313|     417530|
|   Stormbane321|     404310|
|   Battle Drunk|     333693|
|     HD DrAsTiC|     246044|
|    Mario Chief|     239525|
|       Dope Boy|     229047|
| TD Scavenger 7|     222977|
|       Attack3d|     211862|
|      My Regret|     201125|
|   Subject TOAD|     185068|
|         Draper|     181405|
|   LSH Iron Man|     178847|
|   RWB Team USA|     163029|
|  Slippery Smif|     147221|
|   GLoRyLiCiouS|     127306|
+---------------+-----------+
only showing top 20 rows



'EcZachly'

In [39]:
# Which playlist gets played the most?
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

aggregated = df.groupBy("playlist_id").agg(F.count("playlist_id").alias("playlist_count"))
playlist_count = aggregated.orderBy(desc("playlist_count"))
playlist_count.show();

playlist_with_most_plays = playlist_count.first()["playlist_id"] 
playlist_with_most_plays

+--------------------+--------------+
|         playlist_id|playlist_count|
+--------------------+--------------+
|f72e0ef0-7c4a-430...|       1565529|
|780cc101-005c-4fc...|       1116002|
|0bcf2be1-3168-4e4...|       1015496|
|c98949ae-60a8-43d...|        824932|
|2323b76a-db98-4e0...|        692342|
|892189e9-d712-4bd...|        667670|
|f27a65eb-2d11-496...|        167498|
|355dc154-9809-4ed...|        140006|
|d0766624-dbd7-453...|        138470|
|bc0f8ad6-31e6-4a1...|        111073|
|7b7e892c-d9b7-4b0...|         82723|
|7385b4a1-86bf-4ae...|         76425|
|f0c9ef9a-48bd-4b2...|         47813|
|b5d5a242-ffa5-4d8...|         46411|
|819eb188-1a1c-48b...|         39404|
|d21c8381-26f1-4d6...|         37049|
|4b12472e-2a06-423...|         28733|
|5728f612-3f20-445...|         28543|
|0504ca3c-de41-48f...|         22502|
|88b7de19-113c-4be...|         15860|
+--------------------+--------------+
only showing top 20 rows



'f72e0ef0-7c4a-4307-af78-8e38dac3fdba'

In [31]:
# Which map gets played the most?
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

aggregated = df.groupBy("mapid").agg(F.count("mapid").alias("map_count"))
map_count = aggregated.orderBy(desc("map_count"))
map_count.show();

map_with_most_plays = map_count.join(broadcast(maps),"mapid").first()["map_name"]
map_with_most_plays

+--------------------+---------+
|               mapid|map_count|
+--------------------+---------+
|c74c9d0f-f206-11e...|  1445545|
|c7edbf0f-f206-11e...|  1435048|
|c7805740-f206-11e...|   953278|
|cdb934b0-f206-11e...|   396305|
|cb914b9e-f206-11e...|   309045|
|ce1dc2de-f206-11e...|   299736|
|cebd854f-f206-11e...|   298891|
|caacb800-f206-11e...|   291540|
|cd844200-f206-11e...|   261162|
|cc040aa1-f206-11e...|   256966|
|cdee4e70-f206-11e...|   244295|
|c7b7baf0-f206-11e...|   204568|
|ca737f8f-f206-11e...|   156631|
|cbcea2c0-f206-11e...|   135375|
|cc74f4e1-f206-11e...|   132392|
|ce89a40f-f206-11e...|    65081|
+--------------------+---------+



'Plaza'

In [32]:
# Which map do players get the most Killing Spree medals on?
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

killing_spree_medal_id = medals.filter(F.col("name") == "Killing Spree").first()["medal_id"]
killing_spree_medal_id


'2430242797'

In [33]:

aggregated = df.filter(F.col("medal_id") == killing_spree_medal_id).groupBy("mapid").agg(F.count("mapid").alias("map_count"))
map_count = aggregated.orderBy(desc("map_count"))
map_count.show();

map_with_most_killing_spree_medals = map_count.join(broadcast(maps),"mapid").first()["map_name"]
map_with_most_killing_spree_medals

25/11/01 04:00:17 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+--------------------+---------+
|               mapid|map_count|
+--------------------+---------+
|c74c9d0f-f206-11e...|    56908|
|c7edbf0f-f206-11e...|    50570|
|c7805740-f206-11e...|    35444|
|cdb934b0-f206-11e...|    15059|
|ce1dc2de-f206-11e...|    13178|
|cb914b9e-f206-11e...|    12927|
|cebd854f-f206-11e...|    12738|
|caacb800-f206-11e...|    12453|
|cd844200-f206-11e...|    11675|
|cdee4e70-f206-11e...|    11538|
|cc040aa1-f206-11e...|    10918|
|c7b7baf0-f206-11e...|     7259|
|ca737f8f-f206-11e...|     6743|
|cc74f4e1-f206-11e...|     6119|
|cbcea2c0-f206-11e...|     6100|
|ce89a40f-f206-11e...|     2319|
+--------------------+---------+



25/11/01 04:00:19 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


'Plaza'

In [40]:
start_df = df.repartition(16, col("mapid")).withColumn("mapid",col("mapid"))
    
first_sort_df = start_df.sortWithinPartitions(col("mapid"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.map_games_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.map_games_sorted")


In [41]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.map_games_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.map_games_unsorted.files



size,num_files,sorted
55997997,13,sorted
30668445,13,unsorted


In [42]:
start_df = df.repartition(16, col("playlist_id")).withColumn("playlist_id",col("playlist_id"))
    
first_sort_df = start_df.sortWithinPartitions(col("playlist_id"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.playlist_games_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.playlist_games_sorted")


In [44]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.playlist_games_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.playlist_games_unsorted.files



size,num_files,sorted
56987232,12,sorted
30637588,12,unsorted
